#Feature Engineering
This is the Feature Engineering

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [31]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [4]:
path = 'gdrive/MyDrive/DATA/Laptop Price/'

In [5]:
os.listdir(path)

['laptop_price.csv', 'laptop_price_interim.csv']

##Get the Data

In [6]:
df = pd.read_csv(path + 'laptop_price_interim.csv', encoding='latin-1')

In [7]:
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Inches2,Weight2,Ram2,Product2,Company2
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69,13.3,1.37,8.0,MacBook,Apple
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94,13.3,1.34,8.0,Macbook,Apple
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00,15.6,1.86,8.0,250,HP
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45,15.4,1.83,16.0,MacBook,Apple
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60,13.3,1.37,8.0,MacBook,Apple


In [8]:
df.shape

(1303, 18)

In [9]:
df.columns

Index(['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros', 'Inches2', 'Weight2', 'Ram2', 'Product2', 'Company2'],
      dtype='object')

In [10]:
#### predictor
col_num =[ 'Inches2', 'Weight2','Ram2'] ## --> pure number
col_cat =['Company2','Product2','TypeName','OpSys'] ## --> categorical
col_campuran = ['ScreenResolution','Cpu','Memory','Gpu'] ## --> bisa diparsing (dipisah stringnya)
other_col=list(set(df.columns)-set(col_num+col_cat+col_campuran)) ## --> others

In [11]:
other_col

['Company', 'Product', 'Price_euros', 'Weight', 'laptop_ID', 'Inches', 'Ram']

In [12]:
## Predictor --> column2/ feature2 yang digunakan untuk memprediksi

In [13]:
col_target=['Price_euros']

##Train Test

In [14]:
df.shape[0]*0.8

1042.4

In [15]:
id_train = df.sample(1042, random_state=145)['laptop_ID'].values

In [16]:
df['is_train']=df.laptop_ID.isin(id_train).astype(int)

##Feature Campuran

In [17]:
df[col_campuran].sample(8)

,ScreenResolution,Cpu,Memory,Gpu
689,1366x768,Intel Celeron Dual Core 3855U 1.6GHz,500GB HDD,Intel HD Graphics 510
121,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,256GB SSD + 2TB HDD,Nvidia GeForce GTX 1070
1150,1366x768,Intel Core i3 6006U 2.0GHz,1TB HDD,Intel HD Graphics 520
372,IPS Panel Full HD 1920x1080,AMD Ryzen 1700 3GHz,256GB SSD + 1TB HDD,AMD Radeon RX 580
216,1600x900,Intel Core i5 7200U 2.5GHz,1TB HDD,Nvidia GeForce GTX 940MX
540,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 8550U 1.8GHz,256GB SSD,Intel UHD Graphics 620
354,IPS Panel Full HD 1920x1080,Intel Core i7 6500U 2.5GHz,512GB SSD,Nvidia GeForce 930M
393,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,256GB SSD,Nvidia GeForce GTX 1050 Ti


In [18]:
## Memory --> SSD or not
## ScreenResolution --> Full HD, IPS Panel, common
## Cpu --> berapa GHz

In [19]:
def screen_func(x):
  x1=x.lower()
  if 'full hd' in x1:
    return 2
  elif 'ips panel' in x1:
    return 1
  else:
    return 0

In [20]:
df['is_ssd']=df.Memory.apply(lambda x: 1 if 'SSD' in x else 0).astype(int)
df['screen_type']=df.ScreenResolution.apply(screen_func).astype(float)
df['cpu_size']=df.Cpu.apply(lambda x: x[-6:-3].replace(' ','').replace('U2','2').replace('P3','3')).astype(float)

In [21]:
col_num =[ 'Inches2', 'Weight2','Ram2','cpu_size','is_ssd','screen_type'] ## --> pure number
col_cat =['Company2','Product2','TypeName','OpSys'] ## --> categorical

In [22]:
#cat='is_ssd'
#plt.figure(figsize=(15,5))
#sns.boxplot(data = df, y=col_target[0], x=cat)
#plt.show()

##Col Numeric Transformation

In [23]:
XX = df[df.is_train==1][col_num]

SS_scaler = StandardScaler()
SS_scaler.fit(XX)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [60]:
col_num2 =[c+'SS' for c in col_num]
df2_num = pd.DataFrame(SS_scaler.transform(df[col_num]), columns=col_num2)
df2_num['laptop_ID']=df['laptop_ID']
df2_num['is_train']=df['is_train']

In [61]:
col_num2

['Inches2SS', 'Weight2SS', 'Ram2SS', 'cpu_sizeSS', 'is_ssdSS', 'screen_typeSS']

## Col Categoric

In [62]:
col_cat

['Company2', 'Product2', 'TypeName', 'OpSys']

In [63]:
df[col_cat].describe()

,Company2,Product2,TypeName,OpSys
count,1303,1303,1303,1303
unique,9,226,6,9
top,Lenovo,Inspiron,Notebook,Windows 10
freq,297,141,727,1072


In [64]:
col_cat_temp=['Company2',	'TypeName',	'OpSys']
cat = col_cat_temp[1]

In [73]:
col_cat2=[]
df2_cat = []
for cat in col_cat_temp:
  OH = OneHotEncoder()
  XX = df[cat].values.reshape(-1,1)
  OH.fit(XX)

  col_temp = [cat+' '+c for c in OH.categories_]
  df_temp = pd.DataFrame(OH.transform(XX).toarray(), columns= list(col_temp[0]))
  col_cat2 += list(col_temp[0])
  df2_cat.append(df_temp)

In [74]:
df2_cat = pd.concat(df2_cat, axis=1)
df2_cat['laptop_ID'] = df['laptop_ID']
# df2_cat['is_train'] = df['is_train']

## Summary Feature Engineering

In [75]:
col_num2

['Inches2SS', 'Weight2SS', 'Ram2SS', 'cpu_sizeSS', 'is_ssdSS', 'screen_typeSS']

In [76]:
col_cat2

['Company2 Acer',
 'Company2 Apple',
 'Company2 Asus',
 'Company2 Dell',
 'Company2 HP',
 'Company2 Lenovo',
 'Company2 MSI',
 'Company2 Toshiba',
 'Company2 others',
 'TypeName 2 in 1 Convertible',
 'TypeName Gaming',
 'TypeName Netbook',
 'TypeName Notebook',
 'TypeName Ultrabook',
 'TypeName Workstation',
 'OpSys Android',
 'OpSys Chrome OS',
 'OpSys Linux',
 'OpSys Mac OS X',
 'OpSys No OS',
 'OpSys Windows 10',
 'OpSys Windows 10 S',
 'OpSys Windows 7',
 'OpSys macOS']

In [77]:
col_target

['Price_euros']

In [78]:
df2_cat.head()

,Company2 Acer,Company2 Apple,Company2 Asus,Company2 Dell,Company2 HP,Company2 Lenovo,Company2 MSI,Company2 Toshiba,Company2 others,TypeName 2 in 1 Convertible,TypeName Gaming,TypeName Netbook,TypeName Notebook,TypeName Ultrabook,TypeName Workstation,OpSys Android,OpSys Chrome OS,OpSys Linux,OpSys Mac OS X,OpSys No OS,OpSys Windows 10,OpSys Windows 10 S,OpSys Windows 7,OpSys macOS,laptop_ID
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5


In [79]:
df2_num.head()

,Inches2SS,Weight2SS,Ram2SS,cpu_sizeSS,is_ssdSS,screen_typeSS,laptop_ID,is_train
0,-1.231169,-1.014448,-0.087182,0.042451,0.745133,-0.385227,1,1
1,-1.231169,-1.059224,-0.087182,-0.881391,-1.342041,-1.487991,2,1
2,0.398827,-0.283106,-0.087182,0.411987,0.745133,0.717538,3,1
3,0.257089,-0.327882,1.520678,0.781524,0.745133,-0.385227,4,1
4,-1.231169,-1.014448,-0.087182,1.520597,0.745133,-0.385227,5,1


In [81]:
df_interim = pd.merge(df2_num, df2_cat, on ='laptop_ID', how='left')
df_interim = pd.merge(df_interim, df[['laptop_ID']+col_target], on ='laptop_ID', how='left')

In [82]:
df_interim.columns

Index(['Inches2SS', 'Weight2SS', 'Ram2SS', 'cpu_sizeSS', 'is_ssdSS',
       'screen_typeSS', 'laptop_ID', 'is_train', 'Company2 Acer',
       'Company2 Apple', 'Company2 Asus', 'Company2 Dell', 'Company2 HP',
       'Company2 Lenovo', 'Company2 MSI', 'Company2 Toshiba',
       'Company2 others', 'TypeName 2 in 1 Convertible', 'TypeName Gaming',
       'TypeName Netbook', 'TypeName Notebook', 'TypeName Ultrabook',
       'TypeName Workstation', 'OpSys Android', 'OpSys Chrome OS',
       'OpSys Linux', 'OpSys Mac OS X', 'OpSys No OS', 'OpSys Windows 10',
       'OpSys Windows 10 S', 'OpSys Windows 7', 'OpSys macOS', 'Price_euros'],
      dtype='object')

In [83]:
df_interim.to_csv(path + 'data_interim.csv', index=False)

In [84]:
os.listdir(path)

['laptop_price.csv', 'laptop_price_interim.csv', 'data_interim.csv']